In [1]:
from skimage import measure
import collections
import numpy as np
from scipy.spatial import distance
import nibabel as nib
import os
import shutil
import re
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
from pandas.plotting import table 
pd.set_option('display.max_rows', 500)

In [2]:
def getLesionLevelDetectionMetricsV2( reference_image: np.ndarray, predicted_image: np.ndarray) -> collections.namedtuple:
    """

    Lesion-level detection metrics

    Will count TP as predicted lesions that are part of reference rather than
    reference lesions part of prediction (V1 behavior).

    Parameters
    ----------
    reference_image : np.ndarray
        Reference image of zeros (background) and ROIs (above zero).
    predicted_image : np.ndarray
        New image of zeros (background) and ROIs (above zero).

    Returns
    -------
    metrics
        Named tuple of metrics. Get e.g. TP by calling metrics.TP.

    """

    predicted_clusters = measure.label( predicted_image, background=0 )
    true_clusters = measure.label( reference_image, background=0 )

    predicted_overlap = np.multiply(predicted_clusters, reference_image)
    TP = len(np.unique(predicted_overlap))-1 # BG
    FP = np.max(predicted_clusters)-TP

    reference_overlap = np.multiply(true_clusters, predicted_image)
    FN = np.max(true_clusters) - (len(np.unique(reference_overlap))-1)

    P = FN+TP
    numPredClusters = TP+FP

    recall = 0 if P == 0 else TP / P
    precision = 0 if numPredClusters == 0  else TP  / numPredClusters
    f1 = any([precision,recall]) and 2*(precision*recall)/(precision+recall) or 0

    Metrics = collections.namedtuple("Metrics", ["precision", "recall", "f1", "TP", "FP", "FN"])
    return Metrics(precision=precision, recall=recall, f1=f1, TP=TP, FP=FP, FN=FN)

In [3]:
import pandas as pd

def save_to_excel(training_df, subject_df, day_df, results_df, filename):
    with pd.ExcelWriter(filename, engine='xlsxwriter') as writer:
        training_df.to_excel(writer, sheet_name='Training', index=False)
        subject_df.to_excel(writer, sheet_name='Subject', index=False)
        day_df.to_excel(writer, sheet_name='Day', index=False)
        results_df.to_excel(writer, sheet_name='Results', index=False)
    print(f'Data saved to {filename}')



In [4]:


def level_lesion_detection(basepath_label, basepath_pred, training, subjects, days, MRs, vessel1, vessel2):
    results_df = pd.DataFrame()
    array_numbers_df = pd.DataFrame()
    training_df = pd.DataFrame()
    subject_df = pd.DataFrame()
    day_df = pd.DataFrame()
    

    training_segmentations = training_success = 0
    training_segmentations1 = training_success1 = 0
    training_segmentations2 = training_success2 = 0

    for subject in subjects:
        subject_segmentations = subject_success = 0
        subject_segmentations1 = subject_success1 = 0
        subject_segmentations2 = subject_success2 = 0

        for day in days:
            day_segmentations = day_success = 0
            day_segmentations1 = day_success1 = 0
            day_segmentations2 = day_success2 = 0

            for MR in MRs:
                path_label = f'{basepath_label}{subject}_{MR}{day}.nii.gz'
                path_pred = f'{basepath_pred}{training}/{subject}_{MR}{day}.nii.gz'

                if not os.path.exists(path_label):
                    print (f'File not found: {path_label}')
                    continue

                AB = 0

                training_segmentations = 2
                subject_segmentations = 2
                day_segmentations = 2

                training_segmentations1 = 1
                subject_segmentations1 = 1
                day_segmentations1 = 1

                training_segmentations2 = 1
                subject_segmentations2 = 1
                day_segmentations2 = 1

                img_label = nib.load(path_label)
                img_pred = nib.load(path_pred)

                y_true1 = (img_label.get_fdata() == 1)
                y_true2 = (img_label.get_fdata() == 2)
                y_pred1 = (img_pred.get_fdata() == 1)
                y_pred2 = (img_pred.get_fdata() == 2)

                precision1, recall1, f11, TP1, FP1, FN1 = getLesionLevelDetectionMetricsV2(y_true1, y_pred1)
                precision2, recall2, f12, TP2, FP2, FN2 = getLesionLevelDetectionMetricsV2(y_true2, y_pred2)

                training_success = 0
                subject_success = 0
                day_success = 0

                if precision1 == 1 and f11 == 1 and TP1 == 1 and FP1 == 0 and FN1 == 0:
                    training_success += 1
                    subject_success += 1
                    day_success += 1

                    training_success1 = 1
                    subject_success1 = 1
                    day_success1 = 1
                    AB += 1
                else:

                    training_success1 = 0
                    subject_success1 = 0
                    day_success1 = 0

                if precision2 == 1 and f12 == 1 and TP2 == 1 and FP2 == 0 and FN2 == 0:
                    training_success += 1
                    subject_success += 1
                    day_success += 1

                    training_success2 = 1
                    subject_success2 = 1
                    day_success2 = 1
                    AB += 1
                else:
                    training_success2 = 0
                    subject_success2 = 0
                    day_success2 = 0

                temp2_df = pd.DataFrame({
                    f'Training': [training],
                    f'Training Segmentation': [training_segmentations],
                    f'Training Success': [training_success],
                    f'Training Rate': [training_success / training_segmentations if training_segmentations else 0],
                    f'Training Segmentation {vessel1}': [training_segmentations1],
                    f'Training Success {vessel1}': [training_success1],
                    f'Training Rate {vessel1}': [training_success1 / training_segmentations1 if training_segmentations1 else 0],
                    f'Training Segmentation {vessel2}': [training_segmentations2],
                    f'Training Success {vessel2}': [training_success2],
                    f'Training Rate {vessel2}': [training_success2 / training_segmentations2 if training_segmentations2 else 0],
                    f'Training PPV {vessel1}': [precision1],
                    f'Training f1 {vessel1}': [f11],
                    f'Training TPR {vessel1}': [recall1],
                    f'Training PPV {vessel2}': [precision2],
                    f'Training f1 {vessel2}': [f12],
                    f'Training TPR {vessel2}': [recall2],
                    f'Subject': [subject],
                    f'Subject Segmentation': [subject_segmentations],
                    f'Subject Success': [subject_success],
                    f'Subject Rate': [subject_success / subject_segmentations if subject_segmentations else 0],
                    f'Subject Segmentation {vessel1}': [subject_segmentations1],
                    f'Subject Success {vessel1}': [subject_success1],
                    f'Subject Rate {vessel1}': [subject_success1 / subject_segmentations1 if subject_segmentations1 else 0],
                    f'Subject Segmentation {vessel2}': [subject_segmentations2],
                    f'Subject Success {vessel2}': [subject_success2],
                    f'Subject Rate {vessel2}': [subject_success2 / subject_segmentations2 if subject_segmentations2 else 0],
                    f'Day': [day],
                    f'Day Segmentation': [day_segmentations],
                    f'Day Success': [day_success],
                    f'Day Rate': [day_success / day_segmentations if day_segmentations else 0],
                    f'Day Segmentation {vessel1}': [day_segmentations1],
                    f'Day Success {vessel1}': [day_success1],
                    f'Day Rate {vessel1}': [day_success1 / day_segmentations1 if day_segmentations1 else 0],
                    f'Day Segmentation {vessel2}': [day_segmentations2],
                    f'Day Success {vessel2}': [day_success2],
                    f'Day Rate {vessel2}': [day_success2 / day_segmentations2 if day_segmentations2 else 0]
                })

                array_numbers_df = pd.concat([array_numbers_df, temp2_df], ignore_index=True)

                if AB == 2:
                    continue

                temp_df = pd.DataFrame({
                    'Training': [training],
                    'Subject': subject,
                    'Day': day,
                    'MR': MR,
                    f'Pre {vessel1}': [precision1],
                    f'f1 {vessel1}': [f11],
                    f'TP {vessel1}': [TP1],
                    f'FP {vessel1}': [FP1],
                    f'FN {vessel1}': [FN1],
                    f'Pre {vessel2}': [precision2],
                    f'f1 {vessel2}': [f12],
                    f'TP {vessel2}': [TP2],
                    f'FP {vessel2}': [FP2],
                    f'FN {vessel2}': [FN2]
                })
                results_df = pd.concat([results_df, temp_df], ignore_index=True)

            if 'Day' in array_numbers_df.columns:
                tempdf2 = pd.DataFrame({
                    'Training': [training],
                    'Subject': [subject],
                    'Day': [day],
                    'Segmentations': [array_numbers_df[array_numbers_df['Day'] == day]['Day Segmentation'].sum()],
                    'Success': [array_numbers_df[array_numbers_df['Day'] == day]['Day Success'].sum()],
                    'Rate': [array_numbers_df[array_numbers_df['Day'] == day]['Day Rate'].mean()],
                    f'Segmentations {vessel1}': [array_numbers_df[array_numbers_df['Day'] == day][f'Day Segmentation {vessel1}'].sum()],
                    f'Success {vessel1}': [array_numbers_df[array_numbers_df['Day'] == day][f'Day Success {vessel1}'].sum()],
                    f'Rate {vessel1}': [array_numbers_df[array_numbers_df['Day'] == day][f'Day Rate {vessel1}'].mean()],
                    f'Segmentations {vessel2}': [array_numbers_df[array_numbers_df['Day'] == day][f'Day Segmentation {vessel2}'].sum()],
                    f'Success {vessel2}': [array_numbers_df[array_numbers_df['Day'] == day][f'Day Success {vessel2}'].sum()],
                    f'Rate {vessel2}': [array_numbers_df[array_numbers_df['Day'] == day][f'Day Rate {vessel2}'].mean()]
                })
                day_df = pd.concat([day_df, tempdf2], ignore_index=True)

        if 'Subject' in array_numbers_df.columns:
            temp_df3 = pd.DataFrame({
                'Training': [training],
                'Subject': [subject],
                f'Segmentations': [array_numbers_df[array_numbers_df['Subject'] == subject]['Subject Segmentation'].sum()],
                f'Success': [array_numbers_df[array_numbers_df['Subject'] == subject]['Subject Success'].sum()],
                f'Rate': [array_numbers_df[array_numbers_df['Subject'] == subject]['Subject Rate'].mean()],
                f'Segmentations {vessel1}': [array_numbers_df[array_numbers_df['Subject'] == subject][f'Subject Segmentation {vessel1}'].sum()],
                f'Success {vessel1}': [array_numbers_df[array_numbers_df['Subject'] == subject][f'Subject Success {vessel1}'].sum()],
                f'Rate {vessel1}': [array_numbers_df[array_numbers_df['Subject'] == subject][f'Subject Rate {vessel1}'].mean()],
                f'Segmentations {vessel2}': [array_numbers_df[array_numbers_df['Subject'] == subject][f'Subject Segmentation {vessel2}'].sum()],
                f'Success {vessel2}': [array_numbers_df[array_numbers_df['Subject'] == subject][f'Subject Success {vessel2}'].sum()],
                f'Rate {vessel2}': [array_numbers_df[array_numbers_df['Subject'] == subject][f'Subject Rate {vessel2}'].mean()]
            })
            subject_df = pd.concat([subject_df, temp_df3], ignore_index=True)

    if 'Training' in array_numbers_df.columns:
        temp_df1 = pd.DataFrame({
            f'Training': [training],
            f'Segmentations': [array_numbers_df[array_numbers_df['Training'] == training]['Training Segmentation'].sum()],
            f'Success': [array_numbers_df[array_numbers_df['Training'] == training]['Training Success'].sum()],
            f'Rate': [array_numbers_df[array_numbers_df['Training'] == training]['Training Rate'].mean()],
            f'Segmentations {vessel1}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training Segmentation {vessel1}'].sum()],
            f'Success {vessel1}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training Success {vessel1}'].sum()],
            f'Rate {vessel1}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training Rate {vessel1}'].mean()],
            f'Segmentations {vessel2}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training Segmentation {vessel2}'].sum()],
            f'Success {vessel2}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training Success {vessel2}'].sum()],
            f'Rate {vessel2}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training Rate {vessel2}'].mean()],
            f'PPV {vessel1}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training PPV {vessel1}'].mean()],
            f'f1 {vessel1}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training f1 {vessel1}'].mean()],
            f'TPR {vessel1}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training TPR {vessel1}'].mean()],
            f'PPV {vessel2}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training PPV {vessel2}'].mean()],
            f'f1 {vessel2}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training f1 {vessel2}'].mean()],
            f'TPR {vessel2}': [array_numbers_df[array_numbers_df['Training'] == training][f'Training TPR {vessel2}'].mean()]
        })

        training_df = pd.concat([training_df, temp_df1], ignore_index=True)

    print(training_df)
    print('--' * 50)
    print(subject_df)
    print('--' * 50)
    print(day_df)
    print('--' * 50)
    print(results_df)

    return training_df, subject_df, day_df, results_df


In [ ]:
#Example usage of the function

# Define base paths for labels and predictions
basepath_label = '/home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/labelsVal/'
basepath_pred = '/home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/imagesVal_pred_3d_fullres'

# Define different training configurations
#trainings = ['', '_24frames'] #, '_32frames', '_512pixels', '_4batch', '_resenc', '_combi', '_PRes', '_finalGLP17']
training = '_24frames'
# Generate day and MR ranges
subjects = ['DM']
days = [chr(day) for day in range(ord('A'), ord('H') + 1)]
MRs = [str(i).zfill(2) for i in range(1, 13)]

vessel1 = 'VP'
vessel2 = 'AH'


for training in ['_24frames', '_32frames', '_512pixels', '_4batch', '_resenc', '_combi', '_PRes', '_finalGLP17']:
    filename=f'/home/klinfys/Desktop/Scripts_nnUNet/SMA{training}_3d.xlsx'
    training_df, subject_df, day_df, results_df = level_lesion_detection(basepath_label, basepath_pred, training, subjects, days, MRs, vessel1, vessel2)
    save_to_excel(training_df, subject_df, day_df, results_df, filename)

In [6]:
#Example usage of the function for testsubjects

# Define base paths for labels and predictions
basepath_label = '/home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/labelsTs/'
basepath_pred = '/home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/imagesTs_pred_3d_fullres'

# Define different training configurations
subjects = ['HL', 'JN']
training = ''

vessel1 = 'SMA'
vessel2 = 'TC'

days = [chr(day) for day in range(ord('A'), ord('H') + 1)]
MRs = [str(i).zfill(2) for i in range(1, 13)]

filename=f'/home/klinfys/Desktop/Scripts_nnUNet/Ts_VH.xlsx'

# training_df, subject_df, day_df, results_df = level_lesion_detection(basepath_label, basepath_pred, training, subjects, days, MRs, vessel1, vessel2)
# save_to_excel(training_df, subject_df, day_df, results_df, filename)

print(results_df)

File not found: /home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/labelsTs/HL_04A.nii.gz
File not found: /home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/labelsTs/HL_07D.nii.gz
File not found: /home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/labelsTs/HL_08D.nii.gz
File not found: /home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/labelsTs/HL_04F.nii.gz
File not found: /home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/labelsTs/HL_05F.nii.gz
File not found: /home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/labelsTs/HL_05H.nii.gz
File not found: /home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/labelsTs/HL_07H.nii.gz
File not found: /home/klinfys/Desktop/nnUNet_raw/Dataset724_MesTruncus3D/labelsTs/JN_05E.nii.gz
  Training  Segmentations  Success      Rate  Segmentations SMA  Success SMA  \
0                     368      323  0.877717                184          181   

   Rate SMA  Segmentations TC  Success TC   Rate TC   PPV SMA    f1 SMA

In [ ]:
#Example usage of the function for gallbladder for testsubjects

# Define base paths for labels and predictions
basepath_label = '/home/klinfys/Desktop/nnUNet_raw/Dataset333_MonaiLabel/labelsTs/'
basepath_pred = '/home/klinfys/Desktop/nnUNet_raw/Dataset333_MonaiLabel/imagesTs_pred_3d_fullres'

# Define different training configurations
subjects = ['HL', 'JN']
training = ''

vessel1 = 'SMA'
vessel2 = 'TC'

days = [chr(day) for day in range(ord('A'), ord('H') + 1)]
MRs = [str(i).zfill(2) for i in range(1, 103)]

filename=f'/home/klinfys/Desktop/Scripts_nnUNet/Ts_Scanner.xlsx'

training_df, subject_df, day_df, results_df = level_lesion_detection(basepath_label, basepath_pred, training, subjects, days, MRs, vessel1, vessel2)
save_to_excel(training_df, subject_df, day_df, results_df, filename)

In [ ]:
#Example usage of the function for GLP-2ant for testsubjects

# Define base paths for labels and predictions
basepath_label = '/home/klinfys/Desktop/nnUNet_raw/Dataset365_GLP2antMT/labelsTs/'
basepath_pred = '/home/klinfys/Desktop/nnUNet_raw/Dataset365_GLP2antMT/imagesTs_pred_3d_fullres'

# Define different training configurations
subjects = ['01', '02']
training = ''

vessel1 = 'SMA'
vessel2 = 'TC'

days = [chr(day) for day in range(ord('A'), ord('A') + 1)]
MRs = [str(i).zfill(2) for i in range(1, 9)]

filename=f'/home/klinfys/Desktop/Scripts_nnUNet/Ts_GLP-2ant.xlsx'

training_df, subject_df, day_df, results_df = level_lesion_detection(basepath_label, basepath_pred, training, subjects, days, MRs, vessel1, vessel2)
save_to_excel(training_df, subject_df, day_df, results_df, filename)

  Training  Segmentations  Success     Rate  Segmentations SMA  Success SMA  \
0                      32       25  0.78125                 16            9   

   Rate SMA  Segmentations TC  Success TC  Rate TC  PPV SMA    f1 SMA  \
0    0.5625                16          16      1.0  0.78125  0.854167   

   TPR SMA  PPV TC  f1 TC  TPR TC  
0      1.0     1.0    1.0     1.0  
----------------------------------------------------------------------------------------------------
  Training Subject  Segmentations  Success    Rate  Segmentations SMA  \
0               01             16        9  0.5625                  8   
1               02             16       16  1.0000                  8   

   Success SMA  Rate SMA  Segmentations TC  Success TC  Rate TC  
0            1     0.125                 8           8      1.0  
1            8     1.000                 8           8      1.0  
----------------------------------------------------------------------------------------------------
  